### Follium HeatMaps

import pandas as pd
import folium


In [1]:
import pandas as pd
import folium

ModuleNotFoundError: No module named 'folium'

In [ ]:
# read data
df = pd.read_csv('train.csv')
df.head()

In [ ]:
df = df.drop(columns=['trip_duration', 'dropoff_datetime'])
df.head()

In [ ]:
df.info()

### Process Data

Generating month column

In [ ]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime, format='%Y-%m-%d %H:%M:%S')
df['month'] = df.pickup_datetime.apply(lambda x: x.month)
df['week'] = df.pickup_datetime.apply(lambda x: x.week)
df['day'] = df.pickup_datetime.apply(lambda x: x.day)
df['hour'] = df.pickup_datetime.apply(lambda x: x.hour)

In [ ]:
df.head()

In [ ]:
df.info()

As you can see we can obtain the month, week, day, and hour from the pickup_datetime column (which also gets converted to become a datetime column instead of object). We will use these columns in the later parts of our analysis.

### Map Visualizations with Folium

Generate the map for NY


In [ ]:
def generateBaseMap(default_location=[40.693943, -73.985880]):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=11)
    return base_map

In [ ]:
base_map = generateBaseMap()
base_map

**Analysis Question:**<br>
The city is planning to install taxi stops, locations throughout the city where people can get picked up/dropped off by cabs and wait for cabs to pick them up. This initiative is aiming to:
1. Reduce the amount of taxis that stops at the bicycle lanes 
2. Making it easier and safer for the citizens of the city to get a cab especially during night time 
3. Allowing taxi cab drivers to know where to go when they want to find riders
4. Allowing potential riders to have a point where they can go to when they are looking for taxis.

You are tasked with suggesting the best locations for these taxi stops, and also generating insights about the pattern of rides amount throughout the day across the city.

In [ ]:
from folium.plugins import HeatMap


We will be using 2 months worth of data for this analysis, May - June 2016

In [ ]:
df_copy = df[df['month'] > 4]
df_copy['count'] = 1

Calculate the number of times a path was travelled using group by

In [ ]:
df_agg = df_copy[['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum()
df_agg.head()

Reset index and convert the values to list

In [ ]:
df_agg = df_agg.reset_index()
df_agg=df_agg.values.tolist()

Generate map and add heatmap

In [ ]:
base_map = generateBaseMap()
HeatMap(data=df_agg, radius=8, max_zoom=13).add_to(base_map)
base_map.save('hm1.html')

Add click for marker to mark potential locations

In [ ]:
base_map.add_child(folium.ClickForMarker(popup='Potential Location'))
base_map.save('hm2.html')